In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp "/content/drive/MyDrive/Альфа Банк.zip" "/content/alpha.zip"

In [ ]:
!unzip alpha.zip

Archive:  alpha.zip
replace Альфа Банк/baseline.ipynb? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls

 alpha.zip   drive   sample_data  'Альфа Банк'


In [ ]:
%cd 'Альфа Банк'

/content/Альфа Банк


In [ ]:
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")
train_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_14,city_type_0,None,...,0.945281,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other}
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_14,city_type_0,None,...,0.946066,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,199998,month_2,-0.156775,-0.204960,-0.125987,-0.156311,channel_code_9,city_14,city_type_0,None,...,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{}
599996,199998,month_3,-0.156776,-0.204960,-0.125995,-0.156312,channel_code_9,city_14,city_type_0,None,...,0.944497,0.384773,-0.165588,0.546889,0.407687,-0.201123,0.250924,0.374540,{α},{}
599997,199999,month_1,-0.156712,-0.204913,-0.125831,-0.156248,channel_code_14,city_1876,city_type_0,index_city_code_195,...,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{}
599998,199999,month_2,-0.156712,-0.204913,-0.125831,-0.156248,channel_code_14,city_1876,city_type_0,index_city_code_195,...,NaN,NaN,-0.165588,NaN,NaN,-0.201123,NaN,NaN,{},{}


In [ ]:
num_cols = train_df.select_dtypes(include='number').columns
cat_cols = train_df.drop(num_cols, axis=1).columns
clusters = pd.read_excel("cluster_weights.xlsx")

## Обучение


In [ ]:
from sklearn.metrics import roc_auc_score

def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

In [ ]:
# from sklearn.model_selection import train_test_split
# import datetime

# cities = []
# probs = []
# for x, y in zip(train_df['index_city_code'].value_counts(), train_df['index_city_code'].value_counts().index):
#   cities.append(y)
#   probs.append(x)

# probs = np.array(probs)/np.sum(probs)

# import random

# random.choices(
#      population=cities,
#      weights=probs,
#      k=1
#  )

# X = train_df.drop(["id", "date", "end_cluster"
# #, "index_city_code"
#                    ], axis=1)
# y = train_df["end_cluster"]

# num_cols = X.select_dtypes(include='number').columns
# cat_cols = X.drop(num_cols, axis=1).columns

# for column in num_cols:
#   X[column] = X[column].fillna(X[column].median())

# for column in cat_cols:
#   if column == 'index_city_code':
#     X[column] = X[column].apply(lambda x: random.choices(
#      population=cities,
#      weights=probs,
#      k=1
#  )[0] if not x else x)
#   else:
#     X[column] = X[column].fillna(X[column].mode()[0])


# X["ogrn_month_num"] = X["ogrn_month"].apply(lambda x: (int(x.split('_')[2]) + 1) if not x is None else x)
# X["ogrn_year_num"] = X["ogrn_year"].apply(lambda x: int(x.split('_')[2]) + 2000 if not (x is None) else x)
# X['ts'] = [datetime.datetime(int(i), int(j), 1).timestamp() if not(pd.isna(i) or pd.isna(j)) else datetime.datetime(11, 5, 1).timestamp() for i, j in zip(np.array(X["ogrn_year_num"]), np.array(X["ogrn_month_num"]))]

# x_train, x_val, y_train, y_val = train_test_split(X, y,
#                                                   test_size=0.2,
#                                                   stratify=y,
#                                                   random_state=42)

In [ ]:
!pip install catboost optuna

In [ ]:
cluster_weights = pd.read_excel("cluster_weights.xlsx")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [ ]:
# import numpy as np
# import optuna
# from optuna.integration import CatBoostPruningCallback
# import catboost as cb
# from sklearn.datasets import load_breast_cancer
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split


# def objective(trial: optuna.Trial) -> float:
#     train_x, valid_x, train_y, valid_y = train_test_split(x_val, y_val, test_size=0.25, stratify=y_val)

#     # le = LabelEncoder()
#     # train_y = le.fit_transform(train_y)
#     # valid_y = le.transform(valid_y)

#     param = {
#         'learning_rate':trial.suggest_uniform("learning_rate", 0.001, 0.3),
#         #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
#         "depth": trial.suggest_int("depth", 1, 12),
#         "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
#         "boosting_type": "Plain",
#         "bootstrap_type": trial.suggest_categorical(
#             "bootstrap_type", ["Bayesian", "Bernoulli"]
#         ),
#         "eval_metric": "TotalF1",
#         "loss_function": "MultiClass",
#         "task_type": "GPU",
#         "devices": "0:1"
#     }

#     if param["bootstrap_type"] == "Bayesian":
#         param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
#     elif param["bootstrap_type"] == "Bernoulli":
#         param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

#     gbm = cb.CatBoostClassifier(**param, )

#     # pruning_callback = CatBoostPruningCallback(trial, "Accuracy")
#     gbm.fit(
#         train_x,
#         train_y,
#         eval_set=[(valid_x, valid_y)],
#         cat_features=list(cat_cols),
#         verbose=0,
#         early_stopping_rounds=100,
#         # callbacks=[pruning_callback],
#     )

#     # pruning_callback.check_pruned()

#     preds = gbm.predict(valid_x)
#     accuracy = accuracy_score(valid_y, preds)
#     return accuracy


# study = optuna.create_study(
#     # pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
#     direction="maximize"
# )
# study.optimize(objective, n_trials=20, timeout=600)

# print("Number of finished trials: {}".format(len(study.trials)))

# print("Best trial:")
# trial = study.best_trial

# print("  Value: {}".format(trial.value))

# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

In [ ]:
# from catboost import CatBoostClassifier

# model = CatBoostClassifier(**{'iterations': 250, 'learning_rate': 0.2418141586121033, 'depth': 12, 'l2_leaf_reg': 0.014620864278662526, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.17991574290710002},
#         task_type="GPU",
#         devices='0:1')

# model.fit(x_train, y_train,
#         cat_features=list(cat_cols),
#         eval_set=(x_val, y_val)
#         )

In [ ]:
# y_pred_proba = model.predict_proba(x_val)

In [ ]:
# cluster_weights = pd.read_excel("cluster_weights.xlsx").set_index("cluster")
# weights_dict = cluster_weights["unnorm_weight"].to_dict()

In [ ]:
# weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

0.9468273742079701

In [ ]:
# importances = model.get_feature_importance(type='PredictionValuesChange')
# feature_importances = pd.Series(importances, index=X.columns).sort_values()

In [ ]:
# plt.figure(figsize=(20, 20))

# plt.barh(feature_importances.index, feature_importances.values)
# plt.title('CatBoost Feature Importance')
# plt.xlabel('Importance')
# plt.ylabel('Features')

# plt.show()

In [ ]:
# features_dict = {x:y for x, y in zip(feature_importances.index, feature_importances.values)}

In [ ]:
# drop_columns = [x[0] for x in list(features_dict.items()) if x[1] < 0.5]

In [ ]:
# len(drop_columns)

63

In [ ]:
# print(drop_columns)

['cnt_days_cred_h_oper_1m', 'cnt_days_cred_g_oper_1m', 'cnt_b_oper_1m', 'cnt_days_cred_f_oper_1m', 'cnt_cred_f_oper_1m', 'sum_cred_f_oper_1m', 'cnt_cred_d_oper_1m', 'cnt_a_oper_1m', 'cnt_cred_d_oper_3m', 'sum_cred_d_oper_1m', 'cnt_days_cred_g_oper_3m', 'cnt_days_cred_f_oper_3m', 'cnt_cred_g_oper_3m', 'cnt_cred_g_oper_1m', 'cnt_deb_d_oper_1m', 'sum_c_oper_1m', 'cnt_cred_f_oper_3m', 'cnt_days_deb_f_oper_1m', 'sum_cred_d_oper_3m', 'sum_cred_f_oper_3m', 'sum_b_oper_3m', 'sum_b_oper_1m', 'cnt_b_oper_3m', 'cnt_c_oper_1m', 'cnt_deb_g_oper_3m', 'cnt_deb_g_oper_1m', 'sum_cred_g_oper_1m', 'cnt_cred_h_oper_3m', 'cnt_deb_h_oper_1m', 'cnt_a_oper_3m', 'cnt_cred_e_oper_1m', 'sum_a_oper_1m', 'cnt_days_cred_e_oper_1m', 'cnt_days_deb_g_oper_1m', 'sum_cred_h_oper_1m', 'cnt_deb_f_oper_1m', 'sum_deb_f_oper_1m', 'sum_deb_h_oper_1m', 'cnt_days_deb_h_oper_1m', 'sum_deb_d_oper_1m', 'sum_deb_d_oper_3m', 'sum_a_oper_3m', 'cnt_days_deb_f_oper_3m', 'sum_deb_h_oper_3m', 'sum_c_oper_3m', 'cnt_deb_h_oper_3m', 'cnt_da

In [ ]:
drop_columns = ['cnt_days_cred_h_oper_1m', 'cnt_days_cred_g_oper_1m', 'cnt_b_oper_1m', 'cnt_days_cred_f_oper_1m', 'cnt_cred_f_oper_1m', 'sum_cred_f_oper_1m', 'cnt_cred_d_oper_1m', 'cnt_a_oper_1m', 'cnt_cred_d_oper_3m', 'sum_cred_d_oper_1m', 'cnt_days_cred_g_oper_3m', 'cnt_days_cred_f_oper_3m', 'cnt_cred_g_oper_3m', 'cnt_cred_g_oper_1m', 'cnt_deb_d_oper_1m', 'sum_c_oper_1m', 'cnt_cred_f_oper_3m', 'cnt_days_deb_f_oper_1m', 'sum_cred_d_oper_3m', 'sum_cred_f_oper_3m', 'sum_b_oper_3m', 'sum_b_oper_1m', 'cnt_b_oper_3m', 'cnt_c_oper_1m', 'cnt_deb_g_oper_3m', 'cnt_deb_g_oper_1m', 'sum_cred_g_oper_1m', 'cnt_cred_h_oper_3m', 'cnt_deb_h_oper_1m', 'cnt_a_oper_3m', 'cnt_cred_e_oper_1m', 'sum_a_oper_1m', 'cnt_days_cred_e_oper_1m', 'cnt_days_deb_g_oper_1m', 'sum_cred_h_oper_1m', 'cnt_deb_f_oper_1m', 'sum_deb_f_oper_1m', 'sum_deb_h_oper_1m', 'cnt_days_deb_h_oper_1m', 'sum_deb_d_oper_1m', 'sum_deb_d_oper_3m', 'sum_a_oper_3m', 'cnt_days_deb_f_oper_3m', 'sum_deb_h_oper_3m', 'sum_c_oper_3m', 'cnt_deb_h_oper_3m', 'cnt_days_deb_h_oper_3m', 'cnt_cred_h_oper_1m', 'city_type', 'sum_deb_g_oper_1m', 'cnt_deb_d_oper_3m', 'sum_cred_g_oper_3m', 'cnt_c_oper_3m', 'cnt_deb_e_oper_1m', 'cnt_days_cred_h_oper_3m', 'cnt_days_cred_e_oper_3m', 'cnt_days_deb_g_oper_3m', 'cnt_deb_f_oper_3m', 'cnt_days_deb_e_oper_1m', 'sum_deb_f_oper_3m', 'sum_cred_h_oper_3m', 'ogrn_year_num', 'sum_deb_g_oper_3m']

In [ ]:
# cool_train = x_train.drop(columns=drop_columns)
# cool_val = x_val.drop(columns=drop_columns)

In [ ]:
# num_cols = cool_train.select_dtypes(include='number').columns
# cat_cols = cool_train.drop(num_cols, axis=1).columns
# clusters = pd.read_excel("cluster_weights.xlsx")

In [ ]:
# from catboost import CatBoostClassifier

# model = CatBoostClassifier(**{'iterations': 1,
#                               'learning_rate': 0.2418141586121033,
#                               'depth': 12,
#                               'l2_leaf_reg': 0.014620864278662526,
#                               'boosting_type': 'Plain',
#                               'bootstrap_type': 'Bernoulli',
#                               'subsample': 0.17991574290710002},
#                                task_type="GPU",
#                                devices='0:1')

# model.fit(cool_train, y_train,
#         cat_features=list(cat_cols),
#         eval_set=(cool_val, y_val)
#         )

In [ ]:
# y_pred_proba = model.predict_proba(cool_val)
# weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

0.9477837239475744

In [ ]:
# importances = model.get_feature_importance(type='PredictionValuesChange')
# feature_importances = pd.Series(importances, index=cool_train.columns).sort_values()

In [ ]:
# plt.figure(figsize=(20, 20))

# plt.barh(feature_importances.index, feature_importances.values)
# plt.title('CatBoost Feature Importance')
# plt.xlabel('Importance')
# plt.ylabel('Features')

# plt.show()

In [ ]:
# x_test = test_df.copy()
# x_test['start_cluster'] = test_df['start_cluster'].fillna(method='ffill')

In [ ]:
# cities = []
# probs = []
# for x, y in zip(x_test['index_city_code'].value_counts(), x_test['index_city_code'].value_counts().index):
#   cities.append(y)
#   probs.append(x)

# probs = np.array(probs)/np.sum(probs)

# import random

# random.choices(
#      population=cities,
#      weights=probs,
#      k=1
#  )

['index_city_code_98']

In [ ]:
# num_cols = x_test.select_dtypes(include='number').columns
# cat_cols = x_test.drop(num_cols, axis=1).columns

# for column in num_cols:
#   x_test[column] = x_test[column].fillna(x_test[column].median())

# for column in cat_cols:
#   if column == 'index_city_code':
#     x_test[column] = x_test[column].apply(lambda x: random.choices(
#      population=cities,
#      weights=probs,
#      k=1
#  )[0] if not x else x)
#   else:
#     x_test[column] = x_test[column].fillna(x_test[column].mode()[0])

In [ ]:
sample_submission_df = pd.read_csv("sample_submission.csv")

In [ ]:
# cool_test = x_test.drop(columns=drop_columns)

In [ ]:
# num_cols_ = cool_test.select_dtypes(include='number').columns

In [ ]:
# last_m_test_df = cool_test[cool_test["date"] == "month_6"]
# cool_cool_test = cool_test[cool_test["date"] == "month_5"]
# last_m_test_df = last_m_test_df.drop(["id", "date"], axis=1)

In [ ]:
# test_pred_proba = model.predict_proba(last_m_test_df)
# test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)

In [ ]:
# test_pred_proba_df['id'] = cool_test['id'].unique()
# test_pred_proba_df = test_pred_proba_df.reindex(columns=['id', *list(test_pred_proba_df.columns[:-1])])

In [ ]:
# test_pred_proba_df = test_pred_proba_df.reindex(columns=['id', *list(test_pred_proba_df.columns[:-1])])

In [ ]:
# test_pred_proba_df

In [ ]:
# sorted_classes = sorted(test_pred_proba_df.columns.to_list())

In [ ]:
# sample_submission_df[sorted_classes] = test_pred_proba_df
# sample_submission_df.to_csv("baseline_submission.csv", index=False)

In [ ]:
params = {
    'iterations': 170,
    'learning_rate': 0.2418141586121033,
    'depth': 12,
    'l2_leaf_reg': 0.014620864278662526,
    'boosting_type': 'Plain',
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.17991574290710002,
    # 'class_weights': class_weights,
    'task_type': "GPU",
    'devices': '0:1'

}
# model = CatBoostClassifier(**{'iterations': 25,
#                               'learning_rate': 0.2418141586121033,
#                               'depth': 12,
#                               'l2_leaf_reg': 0.014620864278662526,
#                               'boosting_type': 'Plain',
#                               'bootstrap_type': 'Bernoulli',
#                               'subsample': 0.17991574290710002},
#                                class_weights=class_weights,
#                                task_type="GPU",
#                                devices='0:1')

# model.fit(cool_train, y_train,
#         cat_features=list(cat_cols),
#         eval_set=(cool_val, y_val)
#         )

In [ ]:
from sklearn.model_selection import train_test_split
import datetime

cities = []
probs = []
for x, z in zip(train_df['index_city_code'].value_counts(), train_df['index_city_code'].value_counts().index):
  cities.append(z)
  probs.append(x)

probs = np.array(probs)/np.sum(probs)

import random

random.choices(
     population=cities,
     weights=probs,
     k=1
 )

X = train_df.drop(["id", "date", "end_cluster"
#, "index_city_code"
                   ], axis=1)
y = train_df["end_cluster"]

num_cols = X.select_dtypes(include='number').columns
cat_cols = X.drop(num_cols, axis=1).columns

for column in num_cols:
  X[column] = X[column].fillna(X[column].median())

for column in cat_cols:
  if column == 'index_city_code':
    X[column] = X[column].apply(lambda x: random.choices(
     population=cities,
     weights=probs,
     k=1
 )[0] if not x else x)
  else:
    X[column] = X[column].fillna(X[column].mode()[0])


X["ogrn_month_num"] = X["ogrn_month"].apply(lambda x: (int(x.split('_')[2]) + 1) if not x is None else x)
X["ogrn_year_num"] = X["ogrn_year"].apply(lambda x: int(x.split('_')[2]) + 2000 if not (x is None) else x)
X['ts'] = [datetime.datetime(int(i), int(j), 1).timestamp() if not(pd.isna(i) or pd.isna(j)) else datetime.datetime(11, 5, 1).timestamp() for i, j in zip(np.array(X["ogrn_year_num"]), np.array(X["ogrn_month_num"]))]

In [ ]:
X = X.drop(columns=drop_columns)

x_train, x_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.2,
                                                  stratify=y,
                                                  random_state=42)

In [ ]:
x_test = test_df.copy()
x_test['start_cluster'] = test_df['start_cluster'].fillna(method='ffill')

cities = []
probs = []
for x, z in zip(x_test['index_city_code'].value_counts(), x_test['index_city_code'].value_counts().index):
  cities.append(z)
  probs.append(x)

probs = np.array(probs)/np.sum(probs)

import random

random.choices(
     population=cities,
     weights=probs,
     k=1
 )

num_cols = x_test.select_dtypes(include='number').columns
cat_cols = x_test.drop(num_cols, axis=1).columns

for column in num_cols:
  x_test[column] = x_test[column].fillna(x_test[column].median())

for column in cat_cols:
  if column == 'index_city_code':
    x_test[column] = x_test[column].apply(lambda x: random.choices(
     population=cities,
     weights=probs,
     k=1
 )[0] if not x else x)
  else:
    x_test[column] = x_test[column].fillna(x_test[column].mode()[0])



x_test["ogrn_month_num"] = x_test["ogrn_month"].apply(lambda x: (int(x.split('_')[2]) + 1) if not x is None else x)
x_test["ogrn_year_num"] = x_test["ogrn_year"].apply(lambda x: int(x.split('_')[2]) + 2000 if not (x is None) else x)
x_test['ts'] = [datetime.datetime(int(i), int(j), 1).timestamp() if not(pd.isna(i) or pd.isna(j)) else datetime.datetime(11, 5, 1).timestamp() for i, j in zip(np.array(x_test["ogrn_year_num"]), np.array(x_test["ogrn_month_num"]))]
cool_test = x_test.drop(columns=drop_columns)

In [ ]:
last_m_test_df = cool_test[cool_test["date"] == "month_6"].drop(["id", "date"], axis=1)
cool_cool_test = cool_test[cool_test["date"] == "month_5"].drop(["id", "date"], axis=1)

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
from sklearn.model_selection import KFold
import xgboost as xgb

def xgb_train(X, y, test_X, params):
    kf = KFold(n_splits=5, shuffle=True, random_state=2017)
    test_pred = []
    train_pred = []
    val_scores = []
    for dev_index, val_index in kf.split(train_df):
        train_x, valid_x = X.loc[dev_index,:], X.loc[val_index,:]
        train_y, valid_y = y[dev_index], y[val_index]
        xgb_submit_data = xgb.DMatrix(test_X)
        dtrain = xgb.DMatrix(train_x, label=train_y)
        dvalid = xgb.DMatrix(valid_x, label=valid_y)
        xgb_submit_data_train = xgb.DMatrix(X)
        # xgb_model = xgb.train(params, xgb_train_data,
                        #   num_boost_round=2000,
                        #   evals= [(xgb_train_data, 'train'), (xgb_val_data, 'valid')],
                        #   early_stopping_rounds=100,
                        #   verbose_eval=500
                        #  )
        xgb_model = xgb.train(params, dtrain)
        pred_test = xgb_model.predict(xgb_submit_data)
        pred_val = xgb_model.predict(dvalid)
        pred_train = xgb_model.predict(xgb_submit_data_train)

        score = weighted_roc_auc(valid_y, pred_val, sorted(list(np.unique(train_y))), weights_dict)
        val_scores.append(score)
        test_pred.append(pred_test)
        train_pred.append(pred_train)
    return val_scores, test_pred, pred_train

In [ ]:
from sklearn.model_selection import KFold
import xgboost as xgb

def catboost_train(X, y, X_test, params):
    kf = KFold(n_splits=3, shuffle=True, random_state=2017)
    pred_test = []
    pred_train = []
    for dev_index, val_index in kf.split(X):
        train_x, valid_x = X.loc[dev_index,:], X.loc[val_index,:]
        train_y, valid_y = y[dev_index], y[val_index]
        num_cols = train_x.select_dtypes(include='number').columns
        cat_cols = train_x.drop(num_cols, axis=1).columns

        model = CatBoostClassifier(**params)
        model.fit(train_x, train_y,
        cat_features=list(cat_cols),
        eval_set=(valid_x, valid_y)
        )
        pred_test.append(model.predict_proba(X_test))
        pred_val.append(model.predict_proba(X))
    return pred_test, pred_train

In [ ]:
pred_test, pred_val = catboost_train(X, y, last_m_test_df, params)

In [ ]:
x = pred_test[0] + pred_test[1] + pred_test[2]
x /= 3

In [ ]:
x

array([[1.18323308e-02, 3.24822404e-03, 4.38267568e-03, ...,
        9.20298518e-05, 8.14982806e-01, 8.86585939e-08],
       [4.01962359e-04, 6.35699746e-01, 3.57367654e-06, ...,
        1.10350333e-05, 3.63138107e-01, 1.29279160e-07],
       [8.19766903e-01, 6.90453314e-05, 6.96623834e-04, ...,
        6.65079679e-04, 5.34961247e-02, 3.01874628e-07],
       ...,
       [1.15405474e-02, 4.99337265e-02, 5.26947955e-03, ...,
        7.99122560e-04, 8.53584035e-01, 4.29612030e-08],
       [8.64456866e-02, 1.20862749e-01, 1.32101701e-04, ...,
        2.29603443e-04, 7.75020961e-01, 7.08937971e-07],
       [7.70836188e-03, 2.08716430e-01, 1.45732182e-03, ...,
        6.94205768e-05, 7.28328688e-01, 4.23637084e-08]])

In [ ]:
test_pred_proba_df = pd.DataFrame(x, columns=classes)
test_pred_proba_df['id'] = cool_test['id'].unique()
test_pred_proba_df = test_pred_proba_df.reindex(columns=['id', *list(test_pred_proba_df.columns[:-1])])
test_pred_proba_df.to_csv('cool_submission.csv', index=False)

##TO DO
1. Пресс качат
2. Анжуманя
3. Бегит
4. Потягиваний
5. Outlier Detection and Handling: Identify and handle outliers in your dataset that may be affecting the model's performance. You can use techniques such as Z-score, IQR (Interquartile Range), or isolation forests to detect and either remove or transform outliers.
6. Стакинг/блендинг бустингов
7. Ну и фичи посоздавать


##XGB

In [ ]:
!pip install xgboost optuna

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_cols = X.select_dtypes(exclude=['float64', 'int64']).columns

encoded_train = X.copy()
encoded_test = last_m_test_df.copy()
for col in cat_cols:
    if col == 'end_cluster':
      break
    lbl = LabelEncoder()
    lbl.fit(list(X[col].values.astype('str')) + list(last_m_test_df[col].values.astype('str')))
    encoded_train[col] = lbl.transform(list(X[col].values.astype('str')))
    encoded_test[col] = lbl.transform(list(last_m_test_df[col].values.astype('str')))

target_encoder = LabelEncoder()
encoded_y = target_encoder.fit_transform(y)

In [ ]:
# import sklearn
# import optuna

# def objective(trial):
#     data, target = encoded_train.copy(), encoded_y.copy()
#     train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25, stratify=target)
#     dtrain = xgb.DMatrix(train_x, label=train_y)
#     dvalid = xgb.DMatrix(valid_x, label=valid_y)

#     param = {
#         "verbosity": 0,
#         "objective": "multi:softprob",
#         # defines booster, gblinear for linear functions.
#         "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
#         # L2 regularization weight.
#         "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#         # L1 regularization weight.
#         "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#         # sampling ratio for training data.
#         "subsample": trial.suggest_float("subsample", 0.2, 1.0),
#         # sampling according to each tree.
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         "num_class" : 17,
#         "tree_method" : "gpu_hist"
#     }

#     if param["booster"] in ["gbtree", "dart"]:
#         # maximum depth of the tree, signifies complexity of the tree.
#         param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
#         # minimum child weight, larger the term more conservative the tree.
#         param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
#         param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
#         # defines how selective algorithm is.
#         param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
#         param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

#     if param["booster"] == "dart":
#         param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
#         param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
#         param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
#         param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

#     bst = xgb.train(param, dtrain)
#     preds = bst.predict(dvalid)
#     pred_labels = np.argmax(preds, axis=1)
#     print(pred_labels)
#     accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
#     return accuracy


# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=100, timeout=600)

# print("Number of finished trials: ", len(study.trials))
# print("Best trial:")
# trial = study.best_trial

# print("  Value: {}".format(trial.value))
# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

In [ ]:
params={ "objective": "multi:softprob",
        'booster': 'dart',
        'lambda': 3.63407580515308e-05,
        'alpha': 0.050581429315105654,
        'subsample': 0.9360140214177146,
        'colsample_bytree': 0.9391425003494083,
        'max_depth': 9,
        'min_child_weight': 10,
        'eta': 0.00018510876069045116,
        'gamma': 6.218469831634942e-05,
        'grow_policy': 'lossguide',
        'sample_type': 'uniform',
        'normalize_type': 'tree',
        'rate_drop': 3.4193249267402867e-07,
        'skip_drop': 0.0030978399596371968,
        "num_class" : 17,
        "device" : "cuda"}

In [ ]:
from sklearn.model_selection import KFold
import xgboost as xgb

def xgb_train(X, y, test_X, params):
    kf = KFold(n_splits=5, shuffle=True, random_state=2017)
    test_pred = []
    train_pred = []
    val_scores = []
    for dev_index, val_index in kf.split(train_df):
        train_x, valid_x = X.loc[dev_index,:], X.loc[val_index,:]
        train_y, valid_y = y[dev_index], y[val_index]
        xgb_submit_data = xgb.DMatrix(test_X)
        dtrain = xgb.DMatrix(train_x, label=train_y)
        dvalid = xgb.DMatrix(valid_x, label=valid_y)
        xgb_submit_data_train = xgb.DMatrix(X)
        # xgb_model = xgb.train(params, xgb_train_data,
                        #   num_boost_round=2000,
                        #   evals= [(xgb_train_data, 'train'), (xgb_val_data, 'valid')],
                        #   early_stopping_rounds=100,
                        #   verbose_eval=500
                        #  )
        xgb_model = xgb.train(params, dtrain)
        pred_test = xgb_model.predict(xgb_submit_data)
        pred_val = xgb_model.predict(dvalid)
        pred_train = xgb_model.predict(xgb_submit_data_train)

        score = weighted_roc_auc(valid_y, pred_val, sorted(list(np.unique(train_y))), weights_dict)
        val_scores.append(score)
        test_pred.append(pred_test)
        train_pred.append(pred_train)
    return val_scores, test_pred, pred_train

In [ ]:
val_scores, test_pred, train_pred = xgb_train(encoded_train, encoded_y, encoded_test, params)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:17:43] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "iterations" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:17:52] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "iterations" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:18:01] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "iterations" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:18:10] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "iterations" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [19:18:20] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "iterations" } are not used.

  warnings

In [ ]:
test_pred

array([[0.05883603, 0.05877538, 0.05878915, ..., 0.05877412, 0.05945024,
        0.05876395],
       [0.05877243, 0.05920893, 0.05876299, ..., 0.05876292, 0.05934289,
        0.05876094],
       [0.05945564, 0.0587684 , 0.05876938, ..., 0.05878258, 0.05881174,
        0.05876278],
       ...,
       [0.05879349, 0.0588019 , 0.05878742, ..., 0.05878458, 0.05951184,
        0.05876647],
       [0.05885357, 0.05888578, 0.05879402, ..., 0.05879043, 0.05915581,
        0.05878725],
       [0.05878088, 0.05879073, 0.05877604, ..., 0.05876962, 0.05961139,
        0.05876475]], dtype=float32)

In [ ]:
test_pred = test_pred[0] + test_pred[1] + test_pred[2]
test_pred /= 3

In [ ]:
sample_submission_df.columns

Index(['id', '{other}', '{}', '{α, β}', '{α, γ}', '{α, δ}', '{α, ε, η}',
       '{α, ε, θ}', '{α, ε, ψ}', '{α, ε}', '{α, η}', '{α, θ}', '{α, λ}',
       '{α, μ}', '{α, π}', '{α, ψ}', '{α}', '{λ}'],
      dtype='object')

In [ ]:
test_pred

array([[0.05883603, 0.05877538, 0.05878915, ..., 0.05877412, 0.05945024,
        0.05876395],
       [0.05877243, 0.05920893, 0.05876299, ..., 0.05876292, 0.05934289,
        0.05876094],
       [0.05945564, 0.0587684 , 0.05876938, ..., 0.05878258, 0.05881174,
        0.05876278],
       ...,
       [0.05879349, 0.0588019 , 0.05878742, ..., 0.05878458, 0.05951184,
        0.05876647],
       [0.05885357, 0.05888578, 0.05879402, ..., 0.05879043, 0.05915581,
        0.05878725],
       [0.05878088, 0.05879073, 0.05877604, ..., 0.05876962, 0.05961139,
        0.05876475]], dtype=float32)

In [ ]:
test_pred_proba_df = pd.DataFrame(test_pred, columns=sample_submission_df.columns[1:])
test_pred_proba_df['id'] = cool_test['id'].unique()
test_pred_proba_df = test_pred_proba_df.reindex(columns=['id', *list(test_pred_proba_df.columns[:-1])])
test_pred_proba_df.to_csv('cool_submission.csv', index=False)

In [ ]:
test_pred_proba_df

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.058836,0.058775,0.058789,0.058850,0.058781,0.058774,0.058769,0.058769,0.058787,0.058799,0.058783,0.058768,0.058769,0.058764,0.058774,0.059450,0.058764
1,200001,0.058772,0.059209,0.058763,0.058764,0.058761,0.058761,0.058761,0.058761,0.058763,0.058771,0.058763,0.058761,0.058761,0.058761,0.058763,0.059343,0.058761
2,200002,0.059456,0.058768,0.058769,0.058861,0.058772,0.058779,0.058767,0.058783,0.058804,0.058797,0.058792,0.058767,0.058765,0.058763,0.058783,0.058812,0.058763
3,200003,0.058782,0.059374,0.058764,0.058765,0.058762,0.058763,0.058762,0.058762,0.058763,0.058791,0.058764,0.058762,0.058762,0.058762,0.058763,0.059135,0.058762
4,200004,0.058868,0.058866,0.058799,0.058813,0.058799,0.058784,0.058784,0.058784,0.058787,0.058801,0.058786,0.058784,0.058794,0.058784,0.058785,0.059199,0.058784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.058798,0.059145,0.058765,0.058768,0.058761,0.058760,0.058760,0.058760,0.058763,0.058761,0.058764,0.058761,0.058760,0.058760,0.058762,0.059390,0.058760
99996,299996,0.058798,0.058789,0.058783,0.058821,0.058768,0.058767,0.058767,0.058766,0.058771,0.058770,0.058776,0.058768,0.058768,0.058766,0.058768,0.059587,0.058766
99997,299997,0.058793,0.058802,0.058787,0.058847,0.058777,0.058767,0.058767,0.058767,0.058783,0.058770,0.058774,0.058768,0.058769,0.058766,0.058785,0.059512,0.058766
99998,299998,0.058854,0.058886,0.058794,0.058815,0.058789,0.058797,0.058787,0.058787,0.058794,0.058802,0.058789,0.058793,0.058791,0.058787,0.058790,0.059156,0.058787


##LGB


In [ ]:
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd

In [ ]:
!pip install lightgbm

  Using cached lightgbm-4.3.0-py3-none-manylinux_2_28_x86_64.whl (3.1 MB)


In [ ]:
data, target = encoded_train.copy(), encoded_y.copy()
train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25, stratify=target)
dtrain = lgb.Dataset(train_x, label=train_y)
lgvalid = lgb.Dataset(valid_x, valid_y)

param = {
     "objective": "multiclass",
      "num_class" : 17,
}
gbm = lgb.train(param, dtrain, valid_sets=[lgvalid])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034245 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5449
[LightGBM] [Info] Number of data points in the train set: 450000, number of used features: 30
[LightGBM] [Info] Start training from score -2.542592
[LightGBM] [Info] Start training from score -1.589919
[LightGBM] [Info] Start training from score -3.980778
[LightGBM] [Info] Start training from score -2.856821
[LightGBM] [Info] Start training from score -4.991160
[LightGBM] [Info] Start training from score -5.882112
[LightGBM] [Info] Start training from score -6.278850
[LightGBM] [Info] Start training from score -7.414884
[LightGBM] [Info] Start training from score -4.701192
[LightGBM] [Info] Start training from score -2.687986
[LightGBM] [Info] Start training from score -4.523488
[LightGBM] [Info] Start training from score -6.53236

In [ ]:
gbm.predict(valid_x)

array([[1.95490177e-02, 2.23705943e-01, 3.33535089e-03, ...,
        9.37302263e-04, 7.05656077e-01, 0.00000000e+00],
       [9.37836347e-03, 8.61498743e-01, 1.32324629e-03, ...,
        8.42274181e-04, 1.16477280e-01, 0.00000000e+00],
       [1.54868080e-02, 1.89459440e-02, 8.65826241e-01, ...,
        1.69223998e-04, 8.78645509e-02, 0.00000000e+00],
       ...,
       [5.97297796e-03, 1.67126256e-02, 4.41358552e-03, ...,
        4.92407443e-04, 9.53527120e-01, 0.00000000e+00],
       [1.53356850e-02, 4.51630772e-01, 3.27439448e-03, ...,
        9.82106133e-04, 4.99920354e-01, 0.00000000e+00],
       [3.09590407e-02, 1.36702799e-01, 1.76476685e-02, ...,
        9.17369679e-04, 7.65191513e-01, 0.00000000e+00]])

In [1]:
import sklearn
import optuna
import lightgbm as lgb

def objective(trial):
    data, target = encoded_train.copy(), encoded_y.copy()
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25, stratify=target)
    dtrain = lgb.Dataset(train_x, label=train_y)
    # dvalid = lgb.Dataset(valid_x, label=valid_y)

    param = {
      "objective": "multiclass",
      "verbosity": 0,
      "boosting_type": "gbdt",
      "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
      "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
      "num_leaves": trial.suggest_int("num_leaves", 2, 256),
      "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
      "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
      "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
      "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
      "num_class" : 17,
    }

    gbm = lgb.train(param, dtrain)
    print('huh')
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

ModuleNotFoundError: No module named 'optuna'

In [ ]:
from sklearn.model_selection import KFold
import lightgbm

def lgb_train(X, y, test_X, params):
    kf = KFold(n_splits=3, shuffle=True, random_state=2017)
    test_pred = []
    train_pred = []
    val_scores = []
    for dev_index, val_index in kf.split(train_df):
        train_x, valid_x = X.loc[dev_index,:], X.loc[val_index,:]
        train_y, valid_y = y[dev_index], y[val_index]

        lgtrain = lightgbm.Dataset(train_x, train_y, categorical_features=list(cat_cols))
        lgvalid = lightgbm.Dataset(valid_x, valid_y, categorical_features=list(cat_cols))


        model = lightgbm.train(params, lgtrain, 2000, valid_sets=[lgvalid], early_stopping_rounds=100, verbose_eval=100)
        pred_test_iter = model.predict(test_X, num_iteration=model.best_iteration)
        pred_train_iter = model.predict(X, num_iteration=model.best_iteration)
        pred_val = model.predict(valid_x)
        # xgb_model = xgb.train(params, xgb_train_data,
                        #   num_boost_round=2000,
                        #   evals= [(xgb_train_data, 'train'), (xgb_val_data, 'valid')],
                        #   early_stopping_rounds=100,
                        #   verbose_eval=500
                        #  )

        score = weighted_roc_auc(valid_y, pred_val, sorted(list(np.unique(train_y))), weights_dict)
        val_scores.append(score)
        test_pred.append(pred_test_iter)
        train_pred.append(pred_train_iter)
    return val_scores, test_pred, train_pred

In [ ]:
scores, test_pred_lgb, train_pred_lgb = lgb_train(X, y, )